In [38]:
import boto3
import json
import os
from functools import reduce
import pprint
from prettytable import PrettyTable
from youtube_transcript_api import YouTubeTranscriptApi
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
from google.oauth2.credentials import Credentials

from phrasefinder import phrasefinder as pf
from sklearn.feature_extraction import stop_words
import time
from tqdm import tqdm
import statistics
from functools import reduce
from tinydb import TinyDB, Query

comprehend = boto3.client(service_name='comprehend', region_name='us-east-1')

def getBestVideoList(keywords, topn=10):
    scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "0"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "./google_credentials/client_secret_754636752811-rmth1g8e3dl144jda8fddh1ihhj413um.apps.googleusercontent.com.json"

    # Get credentials and create an API client
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(
        client_secrets_file, scopes)

    credentials =Credentials(
        None,
        refresh_token="1//0fNppFYz3o7ABCgYIARAAGA8SNwF-L9IrgIZJAKCn9iSH_172SxyT6cA3mMHDlSQ0MTj9MmKTc6zZRnSy1nwMW5kRkl52JYb4jhg",
        token_uri="https://accounts.google.com/o/oauth2/token",
        client_id="754636752811-rmth1g8e3dl144jda8fddh1ihhj413um.apps.googleusercontent.com",
        client_secret="KhUufHmhS8XI0srgpP__cTCr"
    )

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)

    request = youtube.search().list(
        part="snippet",
        maxResults=topn,
        q=keywords,
        relevanceLanguage='en'
    )
    response = request.execute()
    return response['items']

"""     best_content_list = []
    with os.scandir('./data') as entries:
        for entry in entries:
            with open('./data/' + entry.name, 'r') as file:
                best_content_list.append(file.read())
    return best_content_list """

def getKeyPhrases(content_list):
    def insertFrequency(keyPhrase):
        frequency = {'overall': 0, 'min': 0, 'max': 0, 'frequency_by_content': []}
        for content in content_list:
            _freq = content.count(keyPhrase['Text'])
            frequency['overall'] += _freq
            frequency['min'] = _freq if frequency['min'] == 0 else min(frequency['min'], _freq)
            frequency['max'] = max(frequency['max'], _freq)
            frequency['frequency_by_content'].append(_freq)

        frequency['doc_freq'] = 0
        for e in frequency['frequency_by_content']:
            frequency['doc_freq'] += e>0

        frequency['median'] = statistics.median(frequency['frequency_by_content'])
        return {**keyPhrase, 'frequency': frequency}

    def insertNgramFrequency(keyPhrase):

        db = TinyDB('db.json')
        ngramQuery = Query()

         # Set up your query.
        query = keyPhrase['Text']

        ngramData = db.get(ngramQuery.text == query)

        if ngramData is None:
            ngramData = { 'text': query }

            # Optional: set the maximum number of phrases to return.
            options = pf.SearchOptions()
            options.topk = 1

            # Send the request.
            try:
                result = pf.search(pf.Corpus.AMERICAN_ENGLISH, query, options)
                if result.error:
                    print('Request for {} was not successful: {}'.format(query, result.error['message']))
                    return
  
                time.sleep(0.02)
                ngramData={**ngramData, 'volume_count': result.phrases[0].volume_count, 'match_count': result.phrases[0].match_count}
            except Exception as error:
                print('Fatal error for {}: {}'.format(query, error))
                ngramData={**ngramData, 'volume_count': 0, 'match_count': 0}
            
            # cache ngramData
            db.insert(ngramData)
        
        
        return {**keyPhrase, 'ngram_volume_count': ngramData['volume_count'], 'ngram_match_count': ngramData['match_count']}

    def filterPhrases(phrasesByContent):
        phrases = reduce(lambda x,y: x+y, phrasesByContent)                       
        # trim stop words from begining and end
        def stripStopWords(phrase):
            _stop_words = list(stop_words.ENGLISH_STOP_WORDS) + ['he', 'she', 'you', 'me', 'guy', 'guys']
            while True:
                # repetitively trim stop words from the beginning and the end
                modified = False
                for w in _stop_words:
                    if phrase['Text'].startswith(w+' '):
                        phrase['Text'] = phrase['Text'][len(w)+1:]
                        modified = True
                        break
                    elif phrase['Text'].endswith(' '+w):
                        phrase['Text'] = phrase['Text'][:-len(w)-1]
                        modified = True
                        break
                    elif phrase['Text'] == w:
                        phrase['Text'] = ""
                        break
                if not modified:
                    break
            return phrase

        phrases = list(map(stripStopWords, phrases))

        # only keep bigrams
        phrases = list(filter(lambda x: len(x['Text'].split(' ')) >= 1, phrases))

        # remove duplicates
        unique_phrases=[]
        phrase_set = set()
        for phrase in phrases:
            if phrase['Text'] not in phrase_set:
                phrase_set.add(phrase['Text'])
                unique_phrases.append(phrase)

        print('# Unique phrases: {}'.format(len(unique_phrases)))

        # add frequency for each phrase
        unique_phrases = list(map(insertFrequency, unique_phrases))

        # keep phrases showing up at least once in all documents
        unique_phrases = list(filter(lambda x: x['frequency']['min'] >= 0, unique_phrases))

        # phrase should have at least 3 chars
        unique_phrases = list(filter(lambda x: len(x['Text']) >= 3, unique_phrases))

        # phrase should not have weird symbols
        unique_phrases = list(filter(lambda x: all(x.isalpha() or x.isspace() or x == '-' or x =='$' or x.isdigit() for x in x['Text']), unique_phrases))

        print('# phrases: {}'.format(len(unique_phrases)))

        # add ngrams frequency for each phrase
        for i in tqdm(range(len(unique_phrases))):
            unique_phrases[i] = insertNgramFrequency(unique_phrases[i])
        #unique_phrases = list(map(insertNgramFrequency, unique_phrases))

        phrases = unique_phrases

        # sort key phrases by overall frequency
        phrases.sort(reverse=True, key=lambda x: x['frequency']['overall'])

        print(len(phrases))

        # return most frequent top phrases
        return phrases

    # comprehend api requires each content to be less than 5000 bytes
    # see https://docs.aws.amazon.com/comprehend/latest/dg/guidelines-and-limits.html
    content_list_splited = []
    for content in content_list:
        content_list_splited.extend([content[i: i+4000] for i in range(0, len(content), 4000)])
    """ for content in content_list_splited:
        print(len(content))
 """
    def chunks(l, n):
        for i in range(0, len(l), n):
         yield l[i:i + n]

    content_list_splited_chunks = list(chunks(content_list_splited, 25))

    keyPhrasesByContent = []
    entitiesByContent = []
    for chunk in content_list_splited_chunks:
        keyPhrasesResponse = comprehend.batch_detect_key_phrases(TextList=chunk, LanguageCode='en')
        entitiesResponse = comprehend.batch_detect_entities(TextList=chunk, LanguageCode='en')

        keyPhrasesByContent.extend(list(map(lambda x: x['KeyPhrases'], keyPhrasesResponse['ResultList'])))
        entitiesByContent.extend(list(map(lambda x: x['Entities'], entitiesResponse['ResultList'])))

    # return filterPhrases(keyPhrasesByContent), filterPhrases(entitiesByContent)
    return filterPhrases(keyPhrasesByContent)

def getTranscript(video_id):
    print('Getting transcript for '+video_id)
    return YouTubeTranscriptApi.get_transcript(video_id)

In [27]:
keywords = 'How To Buy Your First Rental'
print('Working on keywords: ' + keywords)
video_list = getBestVideoList(keywords)
# video_list = [{'id': {'videoId': 'kqMtDrsc5Pw'}}]
transcript_list = []
for video in video_list:
    try: 
        transcript_list.append(getTranscript(video['id']['videoId']))
    except:
        pass

content_list = []
for transcript in transcript_list:
    content = '. '.join(list(map(lambda x: x['text'], transcript)))
    # print(content)
    # print("\n\n")
    content_list.append(content)

Working on keywords: How To Buy Your First Rental
Getting transcript for bJx7_1rWC6U
Getting transcript for 7TB_eRhSNV4
Getting transcript for u83O2l1QEj4
Getting transcript for nb9gtT-BqRc
Getting transcript for IOqlt3dwb_c
Getting transcript for LxAniFgeCOg
Getting transcript for qiKg3790dtA
Getting transcript for zoCGqNkBsbA
Getting transcript for a5RExfeU4UY
Getting transcript for yv4ZIaiRMZM


In [ ]:
# keyPhrases, entities = getKeyPhrases(content_list)
keyPhrases = getKeyPhrases(content_list)

In [ ]:
import math

total_words = 0
def insertTfidf(phrase):
    # 1.4 M books
    phrase['tfidf'] = (phrase['frequency']['overall'] / total_words) * (1+math.log(1300000/(1+phrase['ngram_volume_count'])))
    return phrase

def printTable(phrases):
    table = PrettyTable()
    table.field_names=['phrase', 'ori_text', 'score', 'overall_freq', 'min_freq', 'max_freq', 'median_freq', 'doc_freq', 'ngram_volume_count', 'ngram_match_count', 'tfidf', 'entity_type']
    for phrase in phrases:
        entity_type = 'N/A'
        if 'Type' in phrase:
            entity_type = phrase['Type']
        table.add_row([phrase['Text'], phrase['ori_text'], phrase['Score'], phrase['frequency']['overall'], phrase['frequency']['min'], phrase['frequency']['max'], phrase['frequency']['median'], phrase['frequency']['doc_freq'], phrase['ngram_volume_count'], phrase['ngram_match_count'], phrase['tfidf'], entity_type])
    print(table)

for content in content_list:
    total_words += len(content.split())

print(len(keyPhrases))
keyPhrases = list(map(insertTfidf, keyPhrases))
# print top keyphrases table
keyPhrases.sort(reverse=True, key=lambda x: x['tfidf'])
printTable(keyPhrases)
#printTable(entities)